In [18]:
from src import sem_functions, city_functions, expert_functions
import importlib

importlib.reload(sem_functions)

from src.sem_functions import *
from src.city_functions import *
from src.expert_functions import *
import numpy as np
import pandas as pd
import pickle
import gzip
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<module 'src.sem_functions' from '/Users/katharina/Documents/PhD/Scripts_Paper_1/label-uncertainty-so2sat/src/sem_functions.py'>

In [19]:
# import csv 

one_hot_16 = pd.read_csv('../data/one_hot_16.csv').to_numpy()
one_hot_c_16 = pd.read_csv('../data/one_hot_c_16.csv')
votes_16 = pd.read_csv('../data/votes_16.csv').to_numpy()

## 1. Full Dataset

In [20]:
# run sem on full dataset

sem_full = sem_fit_var(one_hot_16, K=16, max_iter=20, rtol=1e-3, restarts=5)

iteration 0
Loss: -554658.669029
Loss: -553255.902614
Loss: -552875.325147
better loss on iteration 0: -552875.3251473000
iteration 1
Loss: -554634.621013
Loss: -553405.329578
Loss: -552868.309576
better loss on iteration 1: -552868.3095760166
iteration 2
Loss: -554648.912238
Loss: -553182.846556
Loss: -552835.757891
better loss on iteration 2: -552835.7578910640
iteration 3
Loss: -554634.757099
Loss: -553308.477523
Loss: -552810.603587
better loss on iteration 3: -552810.6035872988
iteration 4
Loss: -554708.202686
Loss: -553315.504772
Loss: -552894.884581


In [21]:
# extract parameters

pi_full = sem_full[1]
theta_full = sem_full[2]
tau_full = e_step_nozerocorrection(one_hot_16, pi_full, theta_full)

## 2. Per City 

In [22]:
city_list = ['berlin', 'cologne', 'london', 'madrid',
             'milan', 'munich', 'paris', 'rome', 'zurich']

In [23]:
# run sem for instances from each city separately

sem_city = [sem_fit_var_per_city(city, data=one_hot_c_16,
                                 K=16, restarts=5, rtol=1e-3, max_iter=20) for city in city_list]

iteration 0
Loss: -84071.601424
Loss: -83693.252366
Loss: -83538.341951
Loss: -83533.057230
better loss on iteration 0: -83533.0572303410
iteration 1
Loss: -84072.256054
Loss: -83687.920996
Loss: -83552.052393
Loss: -83534.931309
iteration 2
Loss: -84073.290123
Loss: -83679.812310
Loss: -83565.660628
Loss: -83539.839231
iteration 3
Loss: -84071.795523
Loss: -83671.784042
Loss: -83555.293054
Loss: -83537.448776
iteration 4
Loss: -84073.926415
Loss: -83666.499348
Loss: -83547.245725
Loss: -83534.908639
iteration 0
Loss: -66833.482216
Loss: -66775.195120
Loss: -66738.403928
better loss on iteration 0: -66738.4039278157
iteration 1
Loss: -66833.209449
Loss: -66774.489583
Loss: -66741.984550
iteration 2
Loss: -66846.000862
Loss: -66775.320776
Loss: -66750.137395
iteration 3
Loss: -66833.177261
Loss: -66776.160713
Loss: -66748.275075
iteration 4
Loss: -66843.159974
Loss: -66771.226592
Loss: -66747.026384
iteration 0
Loss: -112906.177929
Loss: -112874.363952
Loss: -112871.313394
better loss o

## 3. Per Expert

In [24]:
# create expert-specific dataframes: df_1 -> exclude annotations from expert 1

one_hot_experts = expert_data(votes_16, one_hot_16)

df  0  done.
df  1  done.
df  2  done.
df  3  done.
df  4  done.
df  5  done.
df  6  done.
df  7  done.
df  8  done.
df  9  done.
df  10  done.


In [25]:
# run sem for each expert-specific dataframe 

sem_experts = expert_sem(one_hot_experts, K=16, rtol=1e-3, max_iter=20, restarts=5)

exclude expert  0
iteration 0
Loss: -524614.223151
Loss: -524339.019640
better loss on iteration 0: -524339.0196399577
iteration 1
Loss: -524740.788038
Loss: -524299.525006
better loss on iteration 1: -524299.5250062665
iteration 2
Loss: -524637.012183
Loss: -524303.201023
iteration 3
Loss: -524707.953824
Loss: -524187.536788
better loss on iteration 3: -524187.5367879105
iteration 4
Loss: -524696.791959
Loss: -524247.932230
exclude expert  1
iteration 0
Loss: -541844.766157
Loss: -541148.068845
Loss: -540970.151069
better loss on iteration 0: -540970.1510694962
iteration 1
Loss: -541828.199839
Loss: -541133.386859
Loss: -540978.432932
iteration 2
Loss: -541905.391967
Loss: -541253.216849
Loss: -540960.676457
better loss on iteration 2: -540960.6764566137
iteration 3
Loss: -541907.253459
Loss: -541216.107018
Loss: -540937.546437
better loss on iteration 3: -540937.5464369303
iteration 4
Loss: -541882.303844
Loss: -541228.113757
Loss: -540973.864943
exclude expert  2
iteration 0
Loss: -

In [26]:
# save results to lists

pi_experts=[]
theta_experts=[]
tau_experts_org=[]
tau_experts=[]

j=0
for sem in sem_experts:
    pi_experts.append(sem[1])
    theta_experts.append(sem[2])
    tau_experts_org.append(sem[4])
    tau_experts.append(e_step_nozerocorrection(one_hot_experts[j], sem[1], sem[2]))
    j=j+1

In [28]:
# save results to pickle 

with gzip.open('../data/sem_experts.pkl', 'wb') as file:
    pickle.dump(sem_experts, file)

with gzip.open('../data/sem_full.pkl', 'wb') as file:
    pickle.dump(sem_full, file)

with gzip.open('../data/sem_city.pkl', 'wb') as file:
    pickle.dump(sem_city, file)